In [1]:
import os
os.chdir("../")
%pwd

'd:\\PracticeProjects\\NL_Text_Summarization\\NL_TEXT_SUMMARIZER'

In [2]:
# define entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE : str
    ALL_REQUIRED_FILES : list

In [3]:
# Define Config Manager
from NLPTextSummarizer.constants import *
from NLPTextSummarizer.utils.common import read_yaml, create_directories

# 4. Update the configuration manager(./src/project_name/config/configuration.py)
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        #print(CONFIG_FILE_PATH)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 
        create_directories([self.config.artifacts_root])

    
    def get_data_validation_config(self) -> DataValidationConfig:
        # Move config path dict from config.yaml
        config = self.config.data_validation
        # create the directories as mentioned in config.yaml
        create_directories([config.root_dir])
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            ALL_REQUIRED_FILES = config.ALL_REQUIRED_FILES
        )
        return data_validation_config

In [4]:
# Define components
# 5. Update the components (./src/project_name/components/__init__.py)
import os
from NLPTextSummarizer.logging import logger


class DataValidation:
    def __init__(self, config:DataValidationConfig):
        self.config = config
    
    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = None
            all_files = os.listdir(os.path.join("artifacts", "data_ingestion", "samsum_dataset"))
            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status:{validation_status}")
            return validation_status
        
        except Exception as e:
            raise e


In [5]:
# Pipeline component
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config) 
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2024-05-06 21:56:13,457: INFO: common: yaml file: D:\PracticeProjects\NL_Text_Summarization\NL_TEXT_SUMMARIZER\config\config.yaml loaded successfully]
[2024-05-06 21:56:13,459: INFO: common: yaml file: D:\PracticeProjects\NL_Text_Summarization\NL_TEXT_SUMMARIZER\params.yaml loaded successfully]
[2024-05-06 21:56:13,460: INFO: common: created directory at: artifacts]
[2024-05-06 21:56:13,460: INFO: common: created directory at: artifacts/data_validation]
